In [ ]:
from google.colab import files

# This will open a file picker to upload from your PC
uploaded = files.upload()


In [ ]:
import zipfile
import os

# Unzip the uploaded file
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

# Check the contents
print("Files extracted:")
print(os.listdir("dataset"))


In [ ]:
import os
import shutil

source_dir = 'dataset'
emotion_map = {
    '01': 'neutral',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry'
}

# Step 1: Create target emotion folders
for emotion in emotion_map.values():
    os.makedirs(os.path.join(source_dir, emotion), exist_ok=True)

# Step 2: Loop through Actor folders and move files
for actor_folder in os.listdir(source_dir):
    actor_path = os.path.join(source_dir, actor_folder)

    if os.path.isdir(actor_path) and actor_folder.startswith("Actor_"):
        for file in os.listdir(actor_path):
            if file.endswith('.wav'):
                emotion_code = file.split("-")[2]  # 3rd part of filename
                if emotion_code in emotion_map:
                    emotion = emotion_map[emotion_code]
                    src = os.path.join(actor_path, file)
                    dst = os.path.join(source_dir, emotion, file)
                    shutil.move(src, dst)

print("✅ Audio files sorted into: happy, sad, angry, neutral.")


In [ ]:
for emotion in ['happy', 'sad', 'angry', 'neutral']:
    print(f"{emotion} →", len(os.listdir(f'dataset/{emotion}')), "files")


In [ ]:
dataset_path = 'dataset'


In [ ]:
!pip install librosa soundfile scikit-learn


In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [ ]:
def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, duration=3, offset=0.5)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print("Error:", file_path, e)
        return None


In [ ]:
data_path = 'dataset'

X = []
y = []

for emotion in os.listdir(data_path):
    folder_path = os.path.join(data_path, emotion)
    if not os.path.isdir(folder_path):
        continue
    for file in os.listdir(folder_path):
        if file.endswith('.wav'):
            file_path = os.path.join(folder_path, file)
            features = extract_features(file_path)
            if features is not None:
                X.append(features)
                y.append(emotion)


In [ ]:
print("Total samples:", len(X))
print("Feature shape of one sample:", len(X[0]))
print("Emotion labels:", set(y))


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
from google.colab import files
uploaded = files.upload()



In [ ]:
import librosa
import numpy as np

def extract_features(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, duration=3, offset=0.5)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        print("Error:", file_path, e)
        return None


In [ ]:
test_file = 'myvoice(2).wav'  # Make sure this is your uploaded file
features = extract_features(test_file)

# Check if features were extracted correctly
if features is not None and len(features) == 40:
    features = np.array(features).reshape(1, -1)
    predicted_emotion = model.predict(features)
    print("Predicted Emotion:", predicted_emotion[0])
else:
    print("Feature extraction failed or file is invalid.")
